In [1]:
from torch.utils.data import Dataset
import torchvision.transforms.functional as F
from utils.DiceLoss import DiceLoss
from utils.JaccardIndex import JaccardIndex
import glob
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.feature import blob_doh, blob_log
from skimage.morphology import dilation, square
from skimage.measure import label, regionprops
import math
import sys
# sys.setrecursionlimit(512 * 512)

In [2]:
# Incarcare imagini presegmentate
seg_folder = 'G:/MachineLearning/liver-database/validation/'
models = ['DenseNet201', 'Inception', 'ResNet152', 'ResNeXt101', 'UNet', 'Fusion']
# models = ['DenseNet201']
# models = ['UNet', 'Fusion']
seg = np.zeros((len(models), 4776, 512, 512), dtype=bool)
for i in range(len(models)):
    full_path = seg_folder + models[i]
    seg_files = glob.glob(full_path + '/*.ct')
    for j in range(len(seg_files)):
        seg[i, j, :, :] = np.fromfile(seg_files[j], dtype=np.int8).squeeze().reshape([512, 512]).astype(bool)
    print(f"Done loading {models[i]}")

seg = torch.tensor(seg)

# Incarcare ground truth
lits_seg = glob.glob('G:/MachineLearning/liver-database/validation/masks/*.ct')
ground_truth = np.zeros((4776, 512, 512), dtype=bool)
for j in range(len(lits_seg)):
    ground_truth[j, :, :] = np.fromfile(lits_seg[j], dtype=np.float16).squeeze().reshape([512, 512]).astype(bool)
print(f"Done loading ground truth")
ground_truth = torch.tensor(ground_truth)

Done loading DenseNet201
Done loading Inception
Done loading ResNet152
Done loading ResNeXt101
Done loading UNet
Done loading Fusion
Done loading ground truth


## Post-processing the segmentations

In [3]:
def delete_blob(image, x, y):
    if image[x, y] == True:
        image[x, y] = False
        dx = [-1, 0, 1, 0]
        dy = [0, 1, 0, -1]
        for i in range(4):
            image = delete_blob(image, x + dx[i], y + dy[i])
    return image

In [4]:
for i in range(len(models)):
    print(f"----------------- {models[i]} -----------------")
    for j in range(4776):
        slice = np.squeeze(seg[i, j, :, :])
        label_img = label(slice, connectivity=2)
        props = regionprops(label_img)
        if len(props) < 2:
            continue
        props.sort(key=lambda x: x.area, reverse=True)
        main_area = props[0].area
        main_centroid = props[0].centroid
        for p in props:
            distance = math.sqrt((main_centroid[0] - p.centroid[0])**2 + (main_centroid[1] - p.centroid[1])**2)
            if p.area < main_area / 8 or distance > 200 or p.area < 400:
                for px in range(p.coords.shape[0]):
                    slice[p.coords[px, 0], p.coords[px, 1]] = False

        seg[i, j, :, :] = slice

----------------- DenseNet201 -----------------
----------------- Inception -----------------
----------------- ResNet152 -----------------
----------------- ResNeXt101 -----------------
----------------- UNet -----------------
----------------- Fusion -----------------


In [5]:
# Evaluam DICE per case, average dice per slice si Jaccard
dice_loss = DiceLoss()
jaccard = JaccardIndex()

for i in range(len(models)):
    dice = 1.0 - dice_loss(ground_truth, seg[i, :, :, :].squeeze())
    print(f"Dice score average per case for {models[i]}: {dice:.3f}")
print('')

# for i in range(len(models)):
#     dice = 0
#     for s in range(ground_truth.size(dim=0)):
#         dice += (1.0 - dice_loss(ground_truth[s, :, :], seg[i, s, :, :]))
#     dice = dice / ground_truth.size(dim=0)
#     print(f"Dice score per slice average for {models[i]}: {dice:.3f}")
# print('')

for i in range(len(models)):
    jcc_score = jaccard(ground_truth, seg[i, :, :, :].squeeze())
    print(f"Jaccard index average per case for {models[i]}: {jcc_score:.3f}")

Dice score average per case for DenseNet201: 0.953
Dice score average per case for Inception: 0.950
Dice score average per case for ResNet152: 0.956
Dice score average per case for ResNeXt101: 0.953
Dice score average per case for UNet: 0.950
Dice score average per case for Fusion: 0.957

Jaccard index average per case for DenseNet201: 0.909
Jaccard index average per case for Inception: 0.906
Jaccard index average per case for ResNet152: 0.916
Jaccard index average per case for ResNeXt101: 0.910
Jaccard index average per case for UNet: 0.905
Jaccard index average per case for Fusion: 0.917


In [6]:
# Facem diferentele de segmentari
num_models = len(models)
diff_dice = torch.zeros(num_models, num_models)
for i in range(num_models):
    for j in range(num_models):
        diff_dice[i, j] = 1.0 - dice_loss(seg[i, :, :, :] ^ ground_truth, seg[j, :, :, :] ^ ground_truth)
diff_dice = diff_dice.cpu().numpy()

Text(0.5, 1.0, 'DICE similarity between model segmentations')

In [7]:
plt.figure(figsize = (8, 8))
sns.set(font_scale=1.4)
hmap = sns.heatmap(data=diff_dice, annot=True, xticklabels=models, yticklabels=models, square=True, cbar=False, cmap='viridis')
plt.xticks(rotation=45)
hmap.set_title("DICE similarity between model segmentations", fontsize=20)